In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

2024-03-23 20:01:05.903548: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 20:01:05.943956: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-23 20:01:06.584961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset_path = 'images'
labels = os.listdir(dataset_path)
labels

['test', 'train', 'validation']

In [4]:
train_dir = 'images/train/'
val_dir = 'images/validation/'

# Define emotions
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [5]:
# Count the number of images in each category for the training set
train_counts = []
for emotion in emotions:
    train_counts.append(len(os.listdir(train_dir + emotion)))
print("Training set:")
for i in range(len(emotions)):
    print(emotions[i] + ": " + str(train_counts[i]))

# Count the number of images in each category for the test set
test_counts = []
for emotion in emotions:
    test_counts.append(len(os.listdir(val_dir + emotion)))
print("\nTest set:")
for i in range(len(emotions)):
    print(emotions[i] + ": " + str(test_counts[i]))

Training set:
angry: 6210
disgust: 889
fear: 4687
happy: 12324
neutral: 9547
sad: 9646
surprise: 5228

Test set:
angry: 965
disgust: 116
fear: 1023
happy: 1830
neutral: 1221
sad: 1144
surprise: 802


In [6]:
batch_size = 32
image_size = (224, 224)

datagen_train  = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)

train_set = datagen_train.flow_from_directory(train_dir,
                                              target_size = image_size,
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)


test_set = datagen_val.flow_from_directory(val_dir,
                                              target_size = image_size,
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False)

Found 48531 images belonging to 7 classes.
Found 7101 images belonging to 7 classes.


In [7]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import regularizers
import warnings
warnings.filterwarnings("ignore", message="could not open file to read NUMA node")

In [8]:
# Load the pre-trained VGG16 model
vgg16 = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

2024-03-23 20:01:11.815660: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 20:01:11.856506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 20:01:11.856557: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 20:01:11.861134: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 20:01:11.861202: I external/local_xla/xla/stream_executor

In [9]:
# Freeze the layers of the pre-trained VGG16 model
for layer in vgg16.layers:
    layer.trainable = False

In [10]:
# Add additional layers to the model
x = Flatten()(vgg16.output)
x = Dense(1024, activation='relu')(x)
x = Dense(7, activation='softmax')(x)

In [11]:
model_vgg16 = Model(vgg16.input, x)

In [12]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [13]:
checkpoint = ModelCheckpoint("/vgg16.keras", monitor='val_acc', mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          restore_best_weights=True,
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              min_delta=0.0001)

In [14]:
callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

In [15]:
# Compile the model
optimizer = Adam(learning_rate=1e-4)
model_vgg16.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [16]:
# Train the model
history_1 = model_vgg16.fit(
    train_set,
    epochs=10,
    validation_data=test_set,
    callbacks=callbacks_list,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1711204277.944280   16034 service.cc:145] XLA service 0x7fbfec003660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711204277.944343   16034 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-03-23 20:01:17.975946: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


: 